# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load csv into dataframe
load_csv = "../output_data/cities.csv"
city_df = pd.read_csv(load_csv)
city_df.head()


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,touros,0,BR,1624754136,73,-5.1989,-35.4608,77.00,5.75
1,1,hambantota,20,LK,1624754137,100,6.1241,81.1185,75.54,5.75
2,2,berlevag,100,NO,1624754137,93,70.8578,29.0864,46.40,7.85
3,3,acajutla,100,SV,1624754138,79,13.5928,-89.8275,81.81,1.88
4,4,port alfred,5,CD,1624754139,49,-7.0349,29.7638,69.85,4.59


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = city_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = city_df.loc[(city_df["Wind Speed"] <= 10) & (city_df["Cloudiness"] == 0) & \
                                   (city_df["Max Temp"] >= 70) & (city_df["Max Temp"] <= 80)].dropna()

narrowed_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,touros,0,BR,1624754136,73,-5.1989,-35.4608,77.00,5.75
28,28,sabzevar,0,IR,1624754157,17,36.2126,57.6819,70.90,4.99
38,38,fuengirola,0,ES,1624754164,71,36.5400,-4.6247,75.36,5.75
39,39,sarakhs,0,IR,1624754165,21,36.5449,61.1577,73.54,2.30
75,75,gat,0,IL,1624754192,84,31.6100,34.7642,73.00,1.92
89,89,shahrud,0,IR,1624754203,24,36.4182,54.9763,72.88,7.87
148,148,ziniare,0,BF,1624754249,83,12.5822,-1.2983,75.20,6.91
150,150,chalus,0,IR,1624754250,68,36.6550,51.4204,72.86,0.00
156,156,sagunto,0,ES,1624754254,62,39.6833,-0.2667,73.67,1.14
162,162,aksu,0,CN,1624754258,28,41.1231,80.2644,77.16,4.81


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,touros,BR,-5.1989,-35.4608,
28,sabzevar,IR,36.2126,57.6819,
38,fuengirola,ES,36.5400,-4.6247,
39,sarakhs,IR,36.5449,61.1577,
75,gat,IL,31.6100,34.7642,
89,shahrud,IR,36.4182,54.9763,
148,ziniare,BF,12.5822,-1.2983,
150,chalus,IR,36.6550,51.4204,
156,sagunto,ES,39.6833,-0.2667,
162,aksu,CN,41.1231,80.2644,


In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [7]:
import time
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 0: touros.
Closest hotel in touros is Hotel Vila Gale Touros.
------------
Retrieving Results for Index 28: sabzevar.
Closest hotel in sabzevar is Dinlenme tesisi. Otel.
------------
Retrieving Results for Index 38: fuengirola.
Closest hotel in fuengirola is Higueron Hotel Malaga, Curio Collection by Hilton.
------------
Retrieving Results for Index 39: sarakhs.
Closest hotel in sarakhs is Doosti Hotel.
------------
Retrieving Results for Index 75: gat.
Closest hotel in gat is Desert Gat.
------------
Retrieving Results for Index 89: shahrud.
Closest hotel in shahrud is dorehamy.
------------
Retrieving Results for Index 148: ziniare.
Closest hotel in ziniare is Résidence saints innocent.
------------
Retrieving Results for Index 150: chalus.
Closest hotel in chalus is Hotel khakhamiran.
------------
Retrieving Results for Index 156: sagunto.
Closest hotel in sagunto is Sensity Hotel Vent de Mar.
------------
Retrieving Results for Index 162: aksu.
Closest 

In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))